In [1]:
# Install the necessary library
!pip install --quiet google-generativeai


In [2]:

import os
import json
from datetime import datetime
import google.generativeai as genai

# Set your Gemini API key
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"
genai.configure(api_key=os.environ["GEMINI_API_KEY"])



In [3]:
# Choose your model
MODEL = "gemini-2.5-flash"  # Ensure this model is available in your region
DATA_FILE = "cloudcost_records.json"

# Sample metrics
sample_metrics = [
    {"service": "VM-1", "cpu_usage": 75, "memory_usage": 60, "daily_cost": 12.5},
    {"service": "VM-2", "cpu_usage": 10, "memory_usage": 20, "daily_cost": 8.0},
    {"service": "Storage-1", "usage_gb": 1200, "daily_cost": 5.0}
]


In [4]:


# Function to call Gemini
def ask_gemini(prompt: str):
    model = genai.GenerativeModel(MODEL)
    response = model.generate_content(prompt)
    return response.text



In [5]:
# Function to save records
def save_record(record: dict):
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            try:
                data = json.load(f)
            except:
                data = []
    data.append(record)
    with open(DATA_FILE, "w") as f:
        json.dump(data, f, indent=2)



In [6]:
# Agent orchestration function
def agent_step(metrics):
    system_msg = """
You are CloudCostAI, an intelligent SRE assistant.
Analyze the following metrics, detect anomalies, forecast future cost, and recommend optimizations.
Provide JSON with keys: baseline, forecast, anomalies, recommendations.
"""
    prompt = f"{system_msg}\nMetrics:\n{json.dumps(metrics)}"
    gemini_reply = ask_gemini(prompt)

    try:
        parsed = json.loads(gemini_reply)
    except:
        parsed = {"raw_text": gemini_reply}

    save_record({
        "timestamp": datetime.utcnow().isoformat(),
        "metrics": metrics,
        "agent_output": parsed
    })

    return parsed



In [7]:
# Example usage
sample_metrics = {"cost": [100, 120, 130, 160, 155, 190]}  # example input
output = agent_step(sample_metrics)
print(json.dumps(output, indent=2))


{
  "raw_text": "```json\n{\n  \"baseline\": {\n    \"observation\": \"The cost trend is consistently upward, starting from 100 and reaching 190 over six periods, indicating a clear pattern of increasing expenditure. The average increase per period is approximately 18.\"\n  },\n  \"forecast\": {\n    \"next_period_cost\": 210,\n    \"methodology\": \"Based on a blend of the overall average increase and recent trends (specifically, the average of the last three deltas: +30, -5, +35, resulting in an average increase of +20), the next period's cost is projected to continue its upward trajectory.\"\n  },\n  \"anomalies\": [\n    {\n      \"period_start_value\": 130,\n      \"period_end_value\": 160,\n      \"change\": \"+30\",\n      \"type\": \"Accelerated Growth\",\n      \"description\": \"A significant jump of 30 units from 130 to 160, representing an acceleration in cost growth compared to earlier periods' increases (+20, +10).\"\n    },\n    {\n      \"period_start_value\": 160,\n   

/tmp/ipython-input-3611696044.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
